## Case study  - Uber Data Analysis

You have the uber drive data for a driver which captures the differnet aspects of driving behavior. You are a part of the project where you need to grill this data and report the important findings from the grilling and drilling exercise. You also need to provide some useful insights about the trip behaviour of a Uber driver.
Let us try to dive into the data with pandas and numpy by our side!

#### Dataset - 
There are two datasets to be used here. The dataset contains-

**Uberdrive.csv**
- Trip_Id - Id for the trip
- Start Date - the date and time of the start of the trip
- End Date - the date and time of the end of the trip
- Start Location - staring location of the trip 
- End Location  - location where the trip ended
- Purpose of drive - Purpose of the trip (Business, Personal, Meals, Errands, Meetings, Customer Support etc.)


**Uberdrive_Miles.csv**
- Trip_Id - Id for the trip
- Miles Driven  - Total miles driven between the start and the end of the trip

#### Objective-

The aim is to create a driver profile based on the below aspects on driving behavior.
- Name and number of all the unique start and stop points
- Popular start and stop points
- Rides with same start and stop points
- Starting point from which most miles have been driven
- Start- stop pairs that are most travelled in terms of distance
- busiest month in terms of number of drives and miles driven
- busiest day of the week
- peak hours
- most frequent trip category
- most frequent trip purpose
- miles driven per category and purpose
- percent composition of business miles vs personal miles
- time spent per category and purpose

----------------------
 #### Concepts To cover 
----------------------
- 1. <a href = #link1>Overview of the data at hand</a>
- 2. <a href = #link3>Filtering Data</a> 
- 3. <a href = #link2>Data profiling and the functions offered by pandas for understanding the data</a>
- 4. <a href = #link4>DateTime operations</a> 




In [1]:
# Import the libraries 
import pandas as pd
import numpy as np


### <a id = "link1"></a>Overview of the data

In [2]:
# Read the Data
df= pd.read_csv('uberdrive.csv')

In [3]:
# View first 3 rows of data 
df.head(3)

,Trip_Id,START_DATE*,END_DATE*,CATEGORY*,START*,STOP*,PURPOSE*
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies


In [4]:
df_miles=pd.read_csv('UberDrive_Miles.csv')

In [5]:
df_miles.head(3)

,Trip_Id,MILES*
0,1,5.1
1,2,5.0
2,3,4.8


In [6]:
# understand shape and size of data from Uberdrive
df.shape

(1155, 7)

In [7]:
df.size

8085

In [8]:
# check info about data (includes column names, the number of non-null values in it, and data-type for each column.)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1155 entries, 0 to 1154
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Trip_Id      1155 non-null   int64 
 1   START_DATE*  1155 non-null   object
 2   END_DATE*    1155 non-null   object
 3   CATEGORY*    1155 non-null   object
 4   START*       1155 non-null   object
 5   STOP*        1155 non-null   object
 6   PURPOSE*     653 non-null    object
dtypes: int64(1), object(6)
memory usage: 63.3+ KB


In [9]:
df_miles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1155 entries, 0 to 1154
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Trip_Id  1155 non-null   int64  
 1   MILES*   1155 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 18.2 KB


In [10]:
df_miles.shape

(1155, 2)

1. PURPOSE column has lots of missing values.
2. We can update column names to remove asterisk sign, lets rename the columns.

### Renaming columns

In [11]:
# Approach 1
# Replace the * character from all the  columns
df.columns= df.columns.str.replace('*', '')


<ipython-input-11-ea562ea6473a>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.columns= df.columns.str.replace('*', '')


In [13]:
df.columns #asterisk is removed 

Index(['Trip_Id', 'START_DATE', 'END_DATE', 'CATEGORY', 'START', 'STOP',
       'PURPOSE'],
      dtype='object')

In [12]:
# Approach 2
#
# You can also rename the specific column names 
df_miles=df_miles.rename(columns={'MILES*':'MILES'})

In [14]:
df_miles.columns

Index(['Trip_Id', 'MILES'], dtype='object')

**The column names were updated.**

### <a id = "link3"></a>Filtering dataframes
#### Using null values

In [15]:
#to check if column PURPOSE have null values or not.
df['PURPOSE'].isnull()

0       False
1        True
2       False
3       False
4       False
        ...  
1150    False
1151    False
1152    False
1153    False
1154    False
Name: PURPOSE, Length: 1155, dtype: bool

In [16]:
# shows the top 5 entries where PURPOSE is null
df[df['PURPOSE'].isnull()].head(5)             #isna will give same results

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN
32,33,1/19/2016 9:09,1/19/2016 9:23,Business,Whitebridge,Lake Wellingborough,NaN
85,86,02-09-2016 10:54,02-09-2016 11:07,Personal,Whitebridge,Northwoods,NaN
86,87,02-09-2016 11:43,02-09-2016 11:50,Personal,Northwoods,Tanglewood,NaN
87,88,02-09-2016 13:36,02-09-2016 13:52,Personal,Tanglewood,Preston,NaN



#### Filtering out records based on conditions

In [17]:
# Conditions within dataframe 
#select top 10 entries where miles>30
df_miles[df_miles['MILES']>30].head(10)

,Trip_Id,MILES
4,5,63.7
36,37,40.2
108,109,43.7
232,233,136.0
233,234,30.2
245,246,35.1
246,247,36.5
251,252,57.0
268,269,144.0
269,270,310.3


## <a id = "link2"></a>In a bid to create the driver profile, lets explore the data parameter wise - 

- 1.Destination - (starting and stopping)

- 2.Time - (hour of the day, day of week, month of year)

- 3.Grouping two parameters to get more insights

- 4.Category and Purpose

## 1. Destination
### Understanding  the start and stop points 
###### Through the feature, we will try to understand the below points of the driver profile. 
- Name and number of all the unique start and stop points
- Popular start and stop points
- Rides with same start and stop points
- Starting point from which most miles have been driven
- Start- stop pairs that are most travelled in terms of distance

**Let us handle these one by one**

**1. Name and Number of all unique start and stop points**

In [18]:
# Get the unique starting point, unique destination
# names of unique start points
df.START.unique()

array(['Fort Pierce', 'West Palm Beach', 'Cary', 'Jamaica', 'New York',
       'Elmhurst', 'Midtown', 'East Harlem', 'Flatiron District',
       'Midtown East', 'Hudson Square', 'Lower Manhattan',
       "Hell's Kitchen", 'Downtown', 'Gulfton', 'Houston', 'Eagan Park',
       'Morrisville', 'Durham', 'Farmington Woods', 'Whitebridge',
       'Lake Wellingborough', 'Fayetteville Street', 'Raleigh',
       'Hazelwood', 'Fairmont', 'Meredith Townes', 'Apex', 'Chapel Hill',
       'Northwoods', 'Edgehill Farms', 'Tanglewood', 'Preston',
       'Eastgate', 'East Elmhurst', 'Jackson Heights', 'Long Island City',
       'Katunayaka', 'Unknown Location', 'Colombo', 'Nugegoda',
       'Islamabad', 'R?walpindi', 'Noorpur Shahan', 'Heritage Pines',
       'Westpark Place', 'Waverly Place', 'Wayne Ridge', 'Weston',
       'East Austin', 'West University', 'South Congress', 'The Drag',
       'Congress Ave District', 'Red River District', 'Georgian Acres',
       'North Austin', 'Coxville', 'Conven

In [26]:
df.START.nunique()      #  can use the nunique function

177

In [27]:
# Get the names of stopping destinations, unique destinations
# Names of unique stopping points
df.STOP.unique()

array(['Fort Pierce', 'West Palm Beach', 'Palm Beach', 'Cary',
       'Morrisville', 'New York', 'Queens', 'East Harlem', 'NoMad',
       'Midtown', 'Midtown East', 'Hudson Square', 'Lower Manhattan',
       "Hell's Kitchen", 'Queens County', 'Gulfton', 'Downtown',
       'Houston', 'Jamestown Court', 'Durham', 'Whitebridge',
       'Lake Wellingborough', 'Raleigh', 'Umstead', 'Hazelwood',
       'Westpark Place', 'Meredith Townes', 'Leesville Hollow', 'Apex',
       'Chapel Hill', 'Williamsburg Manor', 'Macgregor Downs',
       'Edgehill Farms', 'Northwoods', 'Tanglewood', 'Preston',
       'Walnut Terrace', 'Jackson Heights', 'East Elmhurst',
       'Midtown West', 'Long Island City', 'Jamaica', 'Unknown Location',
       'Colombo', 'Nugegoda', 'Katunayaka', 'Islamabad', 'R?walpindi',
       'Noorpur Shahan', 'Heritage Pines', 'Waverly Place', 'Wayne Ridge',
       'Depot Historic District', 'Weston', 'West University',
       'South Congress', 'Arts District', 'Congress Ave District

In [28]:
df.STOP.nunique()      # count of unique stopping points

188

**2a. Identify popular start points - top 10**

In [29]:
df['START'].value_counts().head(10)

Cary                201
Unknown Location    148
Morrisville          85
Whitebridge          68
Islamabad            57
Durham               37
Lahore               36
Raleigh              28
Kar?chi              27
Apex                 17
Name: START, dtype: int64

In [30]:
df.START.value_counts().head(10)        #can write column name like this too

Cary                201
Unknown Location    148
Morrisville          85
Whitebridge          68
Islamabad            57
Durham               37
Lahore               36
Raleigh              28
Kar?chi              27
Apex                 17
Name: START, dtype: int64

**2b. Identify popular stop destinations - top 10**

In [31]:
df['STOP'].value_counts().head(10)

Cary                203
Unknown Location    149
Morrisville          84
Whitebridge          65
Islamabad            58
Lahore               36
Durham               36
Raleigh              29
Kar?chi              26
Apex                 17
Name: STOP, dtype: int64

**3.  Are there cases where the start and the stop location are the same  ?**

In [33]:
#df[df['START'].isin(df['STOP'])]
#df.isin({START:STOP})
df[df['START']==df['STOP']]

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting
5,6,01-06-2016 17:15,01-06-2016 17:19,Business,West Palm Beach,West Palm Beach,Meal/Entertain
...,...,...,...,...,...,...,...
1147,1148,12/30/2016 15:41,12/30/2016 16:03,Business,Kar?chi,Kar?chi,Errand/Supplies
1148,1149,12/30/2016 16:45,12/30/2016 17:08,Business,Kar?chi,Kar?chi,Meeting
1149,1150,12/30/2016 23:06,12/30/2016 23:10,Business,Kar?chi,Kar?chi,Customer Visit
1150,1151,12/31/2016 1:07,12/31/2016 1:14,Business,Kar?chi,Kar?chi,Meeting


In [34]:
df[df['START']==df['STOP']].shape

(288, 7)

In [35]:
df[df['START']==df['STOP']].head(5)

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting
5,6,01-06-2016 17:15,01-06-2016 17:19,Business,West Palm Beach,West Palm Beach,Meal/Entertain


**288 trips have the same start and stop points**  

**4.Starting point from which the most miles have been driven**

**In order to use the miles feature, let us now merge the two dataframes so that the all the information is in one dataframe.**
- using merge 


In [38]:
new_df=pd.merge(df,df_miles,how='left',on='Trip_Id')                  #left join is most appropriate
new_df                                     #we don't want to compromise with the bigger dataframe which has more information) 

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain,5.1
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN,5.0
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies,4.8
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting,4.7
4,5,01-06-2016 14:42,01-06-2016 15:49,Business,Fort Pierce,West Palm Beach,Customer Visit,63.7
...,...,...,...,...,...,...,...,...
1150,1151,12/31/2016 1:07,12/31/2016 1:14,Business,Kar?chi,Kar?chi,Meeting,0.7
1151,1152,12/31/2016 13:24,12/31/2016 13:42,Business,Kar?chi,Unknown Location,Temporary Site,3.9
1152,1153,12/31/2016 15:03,12/31/2016 15:38,Business,Unknown Location,Unknown Location,Meeting,16.2
1153,1154,12/31/2016 21:32,12/31/2016 21:50,Business,Katunayake,Gampaha,Temporary Site,6.4


In [39]:
new_df.shape

(1155, 8)

**Let's now use groupby function to find the starting point from which the most miles have been driven**

In [40]:
new_df.groupby(['START'])['MILES'].sum().sort_values(ascending = False ).head(10)

START
Unknown Location    1976.5
Cary                1791.3
Morrisville          671.7
Raleigh              433.0
Islamabad            401.2
Durham               384.4
Jacksonville         375.2
Latta                310.3
Asheville            287.7
Whitebridge          273.4
Name: MILES, dtype: float64

most miles have been driven from Cary

**5. Find the top10 start stop pair that have the most miles covered between them ever.**

In [41]:
# Let us drop the unknown locations
df1=new_df.drop(new_df.loc[new_df['START']=='Unknown Location'].index) 
df1 # Makes a new dataframe, which don't have "Unknown Location" as starting point
           

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain,5.1
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN,5.0
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies,4.8
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting,4.7
4,5,01-06-2016 14:42,01-06-2016 15:49,Business,Fort Pierce,West Palm Beach,Customer Visit,63.7
...,...,...,...,...,...,...,...,...
1149,1150,12/30/2016 23:06,12/30/2016 23:10,Business,Kar?chi,Kar?chi,Customer Visit,0.8
1150,1151,12/31/2016 1:07,12/31/2016 1:14,Business,Kar?chi,Kar?chi,Meeting,0.7
1151,1152,12/31/2016 13:24,12/31/2016 13:42,Business,Kar?chi,Unknown Location,Temporary Site,3.9
1153,1154,12/31/2016 21:32,12/31/2016 21:50,Business,Katunayake,Gampaha,Temporary Site,6.4


In [42]:
df1.shape

(1007, 8)

In [43]:
#alternate approach
#df2 = df[df['START'] != 'Unknown Location']
#df2 = df2[df2['STOP'] != 'Unknown Location']# Further updates the df2 dataframe, by removing "Unknown Location" as stopping point

In [44]:
df2=df1.drop(df1.loc[df['STOP']=='Unknown Location'].index)  #now removing 'unknown location from stopping point'
df2        

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain,5.1
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN,5.0
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies,4.8
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting,4.7
4,5,01-06-2016 14:42,01-06-2016 15:49,Business,Fort Pierce,West Palm Beach,Customer Visit,63.7
...,...,...,...,...,...,...,...,...
1148,1149,12/30/2016 16:45,12/30/2016 17:08,Business,Kar?chi,Kar?chi,Meeting,4.6
1149,1150,12/30/2016 23:06,12/30/2016 23:10,Business,Kar?chi,Kar?chi,Customer Visit,0.8
1150,1151,12/31/2016 1:07,12/31/2016 1:14,Business,Kar?chi,Kar?chi,Meeting,0.7
1153,1154,12/31/2016 21:32,12/31/2016 21:50,Business,Katunayake,Gampaha,Temporary Site,6.4


In [46]:
# top 10 most miles covered between a start stop pair
df2.groupby(['START','STOP'])['MILES'].sum().sort_values(ascending=False).head(10)

START         STOP        
Morrisville   Cary            395.7
Cary          Durham          390.0
              Morrisville     380.0
Raleigh       Cary            365.7
Cary          Raleigh         336.5
Durham        Cary            334.4
Latta         Jacksonville    310.3
Cary          Cary            255.9
Jacksonville  Kissimmee       201.0
Asheville     Mebane          195.9
Name: MILES, dtype: float64

In [47]:
S_S=df2.groupby(['START','STOP'])['MILES'].sum().sort_values(ascending=False).head(10) 
S_S=S_S.reset_index()      #flatten the dataframe        
S_S['Start-Stop']=S_S['START'] + '-' + S_S['STOP']    #new dataframe with start stop pair column
S_S

,START,STOP,MILES,Start-Stop
0,Morrisville,Cary,395.7,Morrisville-Cary
1,Cary,Durham,390.0,Cary-Durham
2,Cary,Morrisville,380.0,Cary-Morrisville
3,Raleigh,Cary,365.7,Raleigh-Cary
4,Cary,Raleigh,336.5,Cary-Raleigh
5,Durham,Cary,334.4,Durham-Cary
6,Latta,Jacksonville,310.3,Latta-Jacksonville
7,Cary,Cary,255.9,Cary-Cary
8,Jacksonville,Kissimmee,201.0,Jacksonville-Kissimmee
9,Asheville,Mebane,195.9,Asheville-Mebane


**The most popular start to destination pair is Morrisville-Cary**

<a id = "link4"></a>
## 2. Start Date - End Date 
### Manipulating date & time objects
#### Lets explore the variables using the below points-
- busiest month in terms of number of drives and miles driven
- busiest day of the week and preferred start hour 
- peak hours

We will create more features for the trip data to be able to cater to above profile mappings

In [48]:
new_df.head(3)

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain,5.1
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN,5.0
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies,4.8


In [49]:
new_df.dtypes

Trip_Id         int64
START_DATE     object
END_DATE       object
CATEGORY       object
START          object
STOP           object
PURPOSE        object
MILES         float64
dtype: object

In [50]:
# Create columns by converting the start and end date into a datetime format
# we can also over write the same column - but for the sake of understanding the difference in formats, we create new columns
new_df['start_dt']=pd.to_datetime(new_df['START_DATE'])
new_df['end_dt']=pd.to_datetime(new_df['END_DATE'])



In [51]:
new_df.dtypes   # See how the dtype is different

Trip_Id                int64
START_DATE            object
END_DATE              object
CATEGORY              object
START                 object
STOP                  object
PURPOSE               object
MILES                float64
start_dt      datetime64[ns]
end_dt        datetime64[ns]
dtype: object

In [52]:
new_df.head(5)           # Print first 5 rows of data.

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES,start_dt,end_dt
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain,5.1,2016-01-01 21:11:00,2016-01-01 21:17:00
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN,5.0,2016-01-02 01:25:00,2016-01-02 01:37:00
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies,4.8,2016-01-02 20:25:00,2016-01-02 20:38:00
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting,4.7,2016-01-05 17:31:00,2016-01-05 17:45:00
4,5,01-06-2016 14:42,01-06-2016 15:49,Business,Fort Pierce,West Palm Beach,Customer Visit,63.7,2016-01-06 14:42:00,2016-01-06 15:49:00


In [53]:
import datetime

In [54]:
datetime.datetime.now()     #first check today's date

datetime.datetime(2021, 9, 30, 11, 53, 54, 320325)

In [55]:
# Create more columns by using the inbuilt functionalities of datetime module  
new_df['start_day']=new_df['start_dt'].dt.day                 # Days encoded as 0-6  ( monday =0, Tue =1 .... )
new_df.head()

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES,start_dt,end_dt,start_day
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain,5.1,2016-01-01 21:11:00,2016-01-01 21:17:00,1
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN,5.0,2016-01-02 01:25:00,2016-01-02 01:37:00,2
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies,4.8,2016-01-02 20:25:00,2016-01-02 20:38:00,2
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting,4.7,2016-01-05 17:31:00,2016-01-05 17:45:00,5
4,5,01-06-2016 14:42,01-06-2016 15:49,Business,Fort Pierce,West Palm Beach,Customer Visit,63.7,2016-01-06 14:42:00,2016-01-06 15:49:00,6


In [56]:
new_df['start_hour']=new_df['start_dt'].dt.hour
new_df.head()

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES,start_dt,end_dt,start_day,start_hour
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain,5.1,2016-01-01 21:11:00,2016-01-01 21:17:00,1,21
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN,5.0,2016-01-02 01:25:00,2016-01-02 01:37:00,2,1
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies,4.8,2016-01-02 20:25:00,2016-01-02 20:38:00,2,20
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting,4.7,2016-01-05 17:31:00,2016-01-05 17:45:00,5,17
4,5,01-06-2016 14:42,01-06-2016 15:49,Business,Fort Pierce,West Palm Beach,Customer Visit,63.7,2016-01-06 14:42:00,2016-01-06 15:49:00,6,14


In [57]:
new_df['start_month']=new_df['start_dt'].dt.month
new_df.head()

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES,start_dt,end_dt,start_day,start_hour,start_month
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain,5.1,2016-01-01 21:11:00,2016-01-01 21:17:00,1,21,1
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN,5.0,2016-01-02 01:25:00,2016-01-02 01:37:00,2,1,1
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies,4.8,2016-01-02 20:25:00,2016-01-02 20:38:00,2,20,1
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting,4.7,2016-01-05 17:31:00,2016-01-05 17:45:00,5,17,1
4,5,01-06-2016 14:42,01-06-2016 15:49,Business,Fort Pierce,West Palm Beach,Customer Visit,63.7,2016-01-06 14:42:00,2016-01-06 15:49:00,6,14,1


In [58]:
new_df['d_of_week']=new_df['start_dt'].dt.weekday
new_df.head()

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES,start_dt,end_dt,start_day,start_hour,start_month,d_of_week
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain,5.1,2016-01-01 21:11:00,2016-01-01 21:17:00,1,21,1,4
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN,5.0,2016-01-02 01:25:00,2016-01-02 01:37:00,2,1,1,5
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies,4.8,2016-01-02 20:25:00,2016-01-02 20:38:00,2,20,1,5
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting,4.7,2016-01-05 17:31:00,2016-01-05 17:45:00,5,17,1,1
4,5,01-06-2016 14:42,01-06-2016 15:49,Business,Fort Pierce,West Palm Beach,Customer Visit,63.7,2016-01-06 14:42:00,2016-01-06 15:49:00,6,14,1,2


In [59]:
new_df['weekday']=new_df['start_dt'].dt.day_name()
new_df.head()

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES,start_dt,end_dt,start_day,start_hour,start_month,d_of_week,weekday
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain,5.1,2016-01-01 21:11:00,2016-01-01 21:17:00,1,21,1,4,Friday
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN,5.0,2016-01-02 01:25:00,2016-01-02 01:37:00,2,1,1,5,Saturday
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies,4.8,2016-01-02 20:25:00,2016-01-02 20:38:00,2,20,1,5,Saturday
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting,4.7,2016-01-05 17:31:00,2016-01-05 17:45:00,5,17,1,1,Tuesday
4,5,01-06-2016 14:42,01-06-2016 15:49,Business,Fort Pierce,West Palm Beach,Customer Visit,63.7,2016-01-06 14:42:00,2016-01-06 15:49:00,6,14,1,2,Wednesday


In [60]:
new_df['cal_month']=new_df['start_dt'].dt.month_name()
new_df.head()

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES,start_dt,end_dt,start_day,start_hour,start_month,d_of_week,weekday,cal_month
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain,5.1,2016-01-01 21:11:00,2016-01-01 21:17:00,1,21,1,4,Friday,January
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN,5.0,2016-01-02 01:25:00,2016-01-02 01:37:00,2,1,1,5,Saturday,January
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies,4.8,2016-01-02 20:25:00,2016-01-02 20:38:00,2,20,1,5,Saturday,January
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting,4.7,2016-01-05 17:31:00,2016-01-05 17:45:00,5,17,1,1,Tuesday,January
4,5,01-06-2016 14:42,01-06-2016 15:49,Business,Fort Pierce,West Palm Beach,Customer Visit,63.7,2016-01-06 14:42:00,2016-01-06 15:49:00,6,14,1,2,Wednesday,January


**Now let us answer the questions below.**
<br>
<br>
**1. Busiest month in terms of number of drives and miles driven**

In [62]:
#groupby calender months and count the number of drives
new_df.groupby(['cal_month'])['Trip_Id'].count().sort_values(ascending=False).head(5)

cal_month
December    146
August      133
November    122
February    115
March       113
Name: Trip_Id, dtype: int64

**December appears to be the busiest month in terms of number of drives**

In [63]:
#groupby calender months and calculate total miles driven
new_df.groupby(['cal_month'])['MILES'].sum().sort_values(ascending=False).head(5)          

cal_month
October    1810.0
March      1693.9
August     1335.5
July       1224.6
April      1113.0
Name: MILES, dtype: float64

**October appears to be the busiest month in terms of miles driven**

**2. Busiest day in terms of number of rides**

In [64]:
# Which day did the driver get most drives? 
new_df.groupby(['weekday'])['Trip_Id'].size().sort_values(ascending=False)         
 
#note that .count() could also have been used. However, .size() makes it look more clean.

weekday
Friday       206
Tuesday      176
Monday       174
Thursday     154
Saturday     150
Sunday       148
Wednesday    147
Name: Trip_Id, dtype: int64

**friday is the busiest day in terms of drives.**

**3. Peak hours ?**

In [65]:
new_df.groupby(['start_hour'])['Trip_Id'].count().sort_values(ascending=False).head() # The number of trips started for each hour.

start_hour
15    98
17    95
18    94
13    94
14    89
Name: Trip_Id, dtype: int64

**Looks like the peak hours seem to be between 3PM - 6PM**

#### Figure out the trips that are starting and ending at the same time (0 minutes elapsed). 
<br>

In [66]:
new_df['diff'] = (new_df['end_dt'] - new_df['start_dt'])
new_df.head()

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES,start_dt,end_dt,start_day,start_hour,start_month,d_of_week,weekday,cal_month,diff
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain,5.1,2016-01-01 21:11:00,2016-01-01 21:17:00,1,21,1,4,Friday,January,0 days 00:06:00
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN,5.0,2016-01-02 01:25:00,2016-01-02 01:37:00,2,1,1,5,Saturday,January,0 days 00:12:00
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies,4.8,2016-01-02 20:25:00,2016-01-02 20:38:00,2,20,1,5,Saturday,January,0 days 00:13:00
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting,4.7,2016-01-05 17:31:00,2016-01-05 17:45:00,5,17,1,1,Tuesday,January,0 days 00:14:00
4,5,01-06-2016 14:42,01-06-2016 15:49,Business,Fort Pierce,West Palm Beach,Customer Visit,63.7,2016-01-06 14:42:00,2016-01-06 15:49:00,6,14,1,2,Wednesday,January,0 days 01:07:00


Trip_Id                  int64
START_DATE              object
END_DATE                object
CATEGORY                object
START                   object
STOP                    object
PURPOSE                 object
MILES                  float64
start_dt        datetime64[ns]
end_dt          datetime64[ns]
start_day                int64
start_hour               int64
start_month              int64
d_of_week                int64
weekday                 object
cal_month               object
diff           timedelta64[ns]
dtype: object

In [67]:
new_df[new_df['diff']=='0 days 00:00:00']   #to check 0 minutes ellapsed

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES,start_dt,end_dt,start_day,start_hour,start_month,d_of_week,weekday,cal_month,diff
751,752,09-06-2016 17:49,09-06-2016 17:49,Business,Unknown Location,Unknown Location,NaN,69.1,2016-09-06 17:49:00,2016-09-06 17:49:00,6,17,9,1,Tuesday,September,0 days
761,762,9/16/2016 7:08,9/16/2016 7:08,Business,Unknown Location,Unknown Location,NaN,1.6,2016-09-16 07:08:00,2016-09-16 07:08:00,16,7,9,4,Friday,September,0 days
798,799,10-08-2016 15:03,10-08-2016 15:03,Business,Karachi,Karachi,NaN,3.6,2016-10-08 15:03:00,2016-10-08 15:03:00,8,15,10,5,Saturday,October,0 days
807,808,10/13/2016 13:02,10/13/2016 13:02,Business,Islamabad,Islamabad,NaN,0.7,2016-10-13 13:02:00,2016-10-13 13:02:00,13,13,10,3,Thursday,October,0 days


**following are the trips that are starting and ending at the same time.**

In [68]:
new_df.dtypes   #to check the datatype of new coumn

Trip_Id                  int64
START_DATE              object
END_DATE                object
CATEGORY                object
START                   object
STOP                    object
PURPOSE                 object
MILES                  float64
start_dt        datetime64[ns]
end_dt          datetime64[ns]
start_day                int64
start_hour               int64
start_month              int64
d_of_week                int64
weekday                 object
cal_month               object
diff           timedelta64[ns]
dtype: object

This creates a timedelta datatype

Find the date time units in  https://docs.scipy.org/doc/numpy/reference/arrays.datetime.html

search for 'Datetime Units'

#### For practice- Exploring existing features to create new ones - Speed 
- Open for all of you to explore and figure out what all can be understood and derived from this feature

In [69]:
datetime.timedelta(seconds=24*60*60).total_seconds()     #understanding the use of total_seconds

86400.0

In [70]:
new_df[:]['diff']/np.timedelta64(1,'s')      #it converts timedelta to float datatype for further calculation

0        360.0
1        720.0
2        780.0
3        840.0
4       4020.0
         ...  
1150     420.0
1151    1080.0
1152    2100.0
1153    1080.0
1154    6180.0
Name: diff, Length: 1155, dtype: float64

In [71]:
new_df['seconds']=new_df[:]['diff']/np.timedelta64(1,'s')         #inserting it as a new column
new_df.head()

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES,start_dt,end_dt,start_day,start_hour,start_month,d_of_week,weekday,cal_month,diff,seconds
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain,5.1,2016-01-01 21:11:00,2016-01-01 21:17:00,1,21,1,4,Friday,January,0 days 00:06:00,360.0
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN,5.0,2016-01-02 01:25:00,2016-01-02 01:37:00,2,1,1,5,Saturday,January,0 days 00:12:00,720.0
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies,4.8,2016-01-02 20:25:00,2016-01-02 20:38:00,2,20,1,5,Saturday,January,0 days 00:13:00,780.0
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting,4.7,2016-01-05 17:31:00,2016-01-05 17:45:00,5,17,1,1,Tuesday,January,0 days 00:14:00,840.0
4,5,01-06-2016 14:42,01-06-2016 15:49,Business,Fort Pierce,West Palm Beach,Customer Visit,63.7,2016-01-06 14:42:00,2016-01-06 15:49:00,6,14,1,2,Wednesday,January,0 days 01:07:00,4020.0


In [72]:
new_df.dtypes

Trip_Id                  int64
START_DATE              object
END_DATE                object
CATEGORY                object
START                   object
STOP                    object
PURPOSE                 object
MILES                  float64
start_dt        datetime64[ns]
end_dt          datetime64[ns]
start_day                int64
start_hour               int64
start_month              int64
d_of_week                int64
weekday                 object
cal_month               object
diff           timedelta64[ns]
seconds                float64
dtype: object

In [73]:
new_df['speed']=new_df['MILES']/new_df['seconds']          #now, making a new column of speed 
new_df.head()

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES,start_dt,end_dt,start_day,start_hour,start_month,d_of_week,weekday,cal_month,diff,seconds,speed
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain,5.1,2016-01-01 21:11:00,2016-01-01 21:17:00,1,21,1,4,Friday,January,0 days 00:06:00,360.0,0.014167
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN,5.0,2016-01-02 01:25:00,2016-01-02 01:37:00,2,1,1,5,Saturday,January,0 days 00:12:00,720.0,0.006944
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies,4.8,2016-01-02 20:25:00,2016-01-02 20:38:00,2,20,1,5,Saturday,January,0 days 00:13:00,780.0,0.006154
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting,4.7,2016-01-05 17:31:00,2016-01-05 17:45:00,5,17,1,1,Tuesday,January,0 days 00:14:00,840.0,0.005595
4,5,01-06-2016 14:42,01-06-2016 15:49,Business,Fort Pierce,West Palm Beach,Customer Visit,63.7,2016-01-06 14:42:00,2016-01-06 15:49:00,6,14,1,2,Wednesday,January,0 days 01:07:00,4020.0,0.015846


In [76]:
new_df['speed_kmph']=new_df['speed']*5793.6384 #converting units of speed from miles per sec to kmph for ease in calculation

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES,start_dt,end_dt,start_day,start_hour,start_month,d_of_week,weekday,cal_month,diff,seconds,speed,speed_kmph
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain,5.1,2016-01-01 21:11:00,2016-01-01 21:17:00,1,21,1,4,Friday,January,0 days 00:06:00,360.0,0.014167,82.076544
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN,5.0,2016-01-02 01:25:00,2016-01-02 01:37:00,2,1,1,5,Saturday,January,0 days 00:12:00,720.0,0.006944,40.233600
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies,4.8,2016-01-02 20:25:00,2016-01-02 20:38:00,2,20,1,5,Saturday,January,0 days 00:13:00,780.0,0.006154,35.653159
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting,4.7,2016-01-05 17:31:00,2016-01-05 17:45:00,5,17,1,1,Tuesday,January,0 days 00:14:00,840.0,0.005595,32.416786
4,5,01-06-2016 14:42,01-06-2016 15:49,Business,Fort Pierce,West Palm Beach,Customer Visit,63.7,2016-01-06 14:42:00,2016-01-06 15:49:00,6,14,1,2,Wednesday,January,0 days 01:07:00,4020.0,0.015846,91.804668
5,6,01-06-2016 17:15,01-06-2016 17:19,Business,West Palm Beach,West Palm Beach,Meal/Entertain,4.3,2016-01-06 17:15:00,2016-01-06 17:19:00,6,17,1,2,Wednesday,January,0 days 00:04:00,240.0,0.017917,103.802688
6,7,01-06-2016 17:30,01-06-2016 17:35,Business,West Palm Beach,Palm Beach,Meeting,7.1,2016-01-06 17:30:00,2016-01-06 17:35:00,6,17,1,2,Wednesday,January,0 days 00:05:00,300.0,0.023667,137.116109
7,8,01-07-2016 13:27,01-07-2016 13:33,Business,Cary,Cary,Meeting,0.8,2016-01-07 13:27:00,2016-01-07 13:33:00,7,13,1,3,Thursday,January,0 days 00:06:00,360.0,0.002222,12.874752
8,9,01-10-2016 08:05,01-10-2016 08:25,Business,Cary,Morrisville,Meeting,8.3,2016-01-10 08:05:00,2016-01-10 08:25:00,10,8,1,6,Sunday,January,0 days 00:20:00,1200.0,0.006917,40.072666
9,10,01-10-2016 12:17,01-10-2016 12:44,Business,Jamaica,New York,Customer Visit,16.5,2016-01-10 12:17:00,2016-01-10 12:44:00,10,12,1,6,Sunday,January,0 days 00:27:00,1620.0,0.010185,59.009280


In [78]:
new_df[new_df['speed_kmph']>150].head(5)  #putting 150kmph as speed limit to check over-speeding cases

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES,start_dt,end_dt,start_day,start_hour,start_month,d_of_week,weekday,cal_month,diff,seconds,speed,speed_kmph
121,122,2/18/2016 8:19,2/18/2016 8:27,Business,Unknown Location,Unknown Location,Temporary Site,23.5,2016-02-18 08:19:00,2016-02-18 08:27:00,18,8,2,3,Thursday,February,0 days 00:08:00,480.0,0.048958,283.646880
335,336,4/28/2016 13:30,4/28/2016 13:49,Business,Raleigh,Cary,Customer Visit,32.8,2016-04-28 13:30:00,2016-04-28 13:49:00,28,13,4,3,Thursday,April,0 days 00:19:00,1140.0,0.028772,166.694157
375,376,5/18/2016 13:00,5/18/2016 13:02,Business,Morrisville,Raleigh,Customer Visit,7.6,2016-05-18 13:00:00,2016-05-18 13:02:00,18,13,5,2,Wednesday,May,0 days 00:02:00,120.0,0.063333,366.930432
751,752,09-06-2016 17:49,09-06-2016 17:49,Business,Unknown Location,Unknown Location,NaN,69.1,2016-09-06 17:49:00,2016-09-06 17:49:00,6,17,9,1,Tuesday,September,0 days 00:00:00,0.0,inf,inf
753,754,09-11-2016 09:51,09-11-2016 09:55,Business,Unknown Location,Unknown Location,NaN,8.6,2016-09-11 09:51:00,2016-09-11 09:55:00,11,9,9,6,Sunday,September,0 days 00:04:00,240.0,0.035833,207.605376


In [79]:
new_df[new_df['speed_kmph']>150].shape

(19, 20)

1.Above result also includes inf speed <br>
2.let's first remove speed which are shown infinite but are actually zero due to no distance travelled.

In [80]:
new_df.replace([np.inf , -np.inf], np.nan, inplace=True)

In [81]:
new_df[new_df['speed_kmph']>150].shape #to check whether inf values are converted successfully or not

(15, 20)

**4 inf values are changed to zero and also
from above data of speed greater than 150, we can see that some anomalies are present as speed can't be more than 200kmph that too in rare cases**

In [82]:
#to replace with a suitable value let's calculate average speed
new_df['speed_kmph'].mean()

43.142808241535754

In [83]:
new_df[new_df['speed_kmph']>150]

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES,start_dt,end_dt,start_day,start_hour,start_month,d_of_week,weekday,cal_month,diff,seconds,speed,speed_kmph
121,122,2/18/2016 8:19,2/18/2016 8:27,Business,Unknown Location,Unknown Location,Temporary Site,23.5,2016-02-18 08:19:00,2016-02-18 08:27:00,18,8,2,3,Thursday,February,0 days 00:08:00,480.0,0.048958,283.646880
335,336,4/28/2016 13:30,4/28/2016 13:49,Business,Raleigh,Cary,Customer Visit,32.8,2016-04-28 13:30:00,2016-04-28 13:49:00,28,13,4,3,Thursday,April,0 days 00:19:00,1140.0,0.028772,166.694157
375,376,5/18/2016 13:00,5/18/2016 13:02,Business,Morrisville,Raleigh,Customer Visit,7.6,2016-05-18 13:00:00,2016-05-18 13:02:00,18,13,5,2,Wednesday,May,0 days 00:02:00,120.0,0.063333,366.930432
753,754,09-11-2016 09:51,09-11-2016 09:55,Business,Unknown Location,Unknown Location,NaN,8.6,2016-09-11 09:51:00,2016-09-11 09:55:00,11,9,9,6,Sunday,September,0 days 00:04:00,240.0,0.035833,207.605376
754,755,09-11-2016 21:40,09-11-2016 21:42,Business,Unknown Location,Unknown Location,NaN,9.8,2016-09-11 21:40:00,2016-09-11 21:42:00,11,21,9,6,Sunday,September,0 days 00:02:00,120.0,0.081667,473.147136
762,763,9/18/2016 18:07,9/18/2016 18:11,Business,Unknown Location,Unknown Location,NaN,9.4,2016-09-18 18:07:00,2016-09-18 18:11:00,18,18,9,6,Sunday,September,0 days 00:04:00,240.0,0.039167,226.917504
773,774,9/27/2016 8:33,9/27/2016 8:35,Business,Unknown Location,Unknown Location,NaN,5.8,2016-09-27 08:33:00,2016-09-27 08:35:00,27,8,9,1,Tuesday,September,0 days 00:02:00,120.0,0.048333,280.025856
781,782,10-03-2016 17:09,10-03-2016 17:12,Business,Unknown Location,Islamabad,NaN,10.5,2016-10-03 17:09:00,2016-10-03 17:12:00,3,17,10,0,Monday,October,0 days 00:03:00,180.0,0.058333,337.962240
786,787,10-04-2016 12:17,10-04-2016 12:18,Business,Unknown Location,Unknown Location,NaN,15.1,2016-10-04 12:17:00,2016-10-04 12:18:00,4,12,10,1,Tuesday,October,0 days 00:01:00,60.0,0.251667,1458.065664
788,789,10-06-2016 17:23,10-06-2016 17:40,Business,R?walpindi,Unknown Location,NaN,112.6,2016-10-06 17:23:00,2016-10-06 17:40:00,6,17,10,3,Thursday,October,0 days 00:17:00,1020.0,0.110392,639.572239


In [84]:
for x in new_df.index:
  if new_df.loc[x, "speed_kmph"] > 150:
    new_df.loc[x, "speed_kmph"] = new_df['speed_kmph'].mean()

In [85]:
new_df[new_df['speed_kmph']>150]

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES,start_dt,end_dt,start_day,start_hour,start_month,d_of_week,weekday,cal_month,diff,seconds,speed,speed_kmph


## 4. Category & Purpose
#### Explore the category and the purpose of the trips through
- Most frequent trip category
- Most frequent trip purpose
- Miles driven per category and purpose
- Percent composition of business miles vs personal miles
- time spent per category and purpose

**1. Most frequent trip category**

In [86]:
new_df.groupby(['CATEGORY'])['Trip_Id'].count()

CATEGORY
Business    1078
Personal      77
Name: Trip_Id, dtype: int64

**Most trips are in the business category**

**2. Most frequent Purpose**

In [87]:
new_df.groupby(['PURPOSE'])['Trip_Id'].count().sort_values(ascending=False)

PURPOSE
Meeting            187
Meal/Entertain     160
Errand/Supplies    128
Customer Visit     101
Temporary Site      50
Between Offices     18
Moving               4
Airport/Travel       3
Charity ($)          1
Commute              1
Name: Trip_Id, dtype: int64

**Most trips are for meetings**

In [88]:
#Average distance traveled for each activity
new_df.groupby(['PURPOSE'])['MILES'].mean().sort_values(ascending=False)

PURPOSE
Commute            180.200000
Customer Visit      20.688119
Meeting             15.247594
Charity ($)         15.100000
Between Offices     10.944444
Temporary Site      10.474000
Meal/Entertain       5.698125
Airport/Travel       5.500000
Moving               4.550000
Errand/Supplies      3.968750
Name: MILES, dtype: float64

**Now lets try to answer some questions from this data.**

**Question3**: How many miles were driven per category and purpose ?

**Question4**: What is percentage of business miles vs personal?

**Question5**: How much time was spend for drives per category and purpose?  - <i>for practice: you will need to create a time difference variable - answers will be shared through a notebook</i>


In [90]:
#Question3: How many miles were driven per category and purpose ?
new_df.groupby(['CATEGORY','PURPOSE'])['MILES'].sum().sort_values(ascending=False)

CATEGORY  PURPOSE        
Business  Meeting            2851.3
          Customer Visit     2089.5
          Meal/Entertain      911.7
          Temporary Site      523.7
          Errand/Supplies     508.0
          Between Offices     197.0
Personal  Commute             180.2
          Moving               18.2
Business  Airport/Travel       16.5
Personal  Charity ($)          15.1
Name: MILES, dtype: float64

In [92]:
#Question4: What is percentage of business miles vs personal?
perc=new_df.groupby(['CATEGORY'])['MILES'].sum()
perc

CATEGORY
Business    11487.0
Personal      717.7
Name: MILES, dtype: float64

In [93]:
DF=perc.to_frame()           #converting above series to dataframe
DF

,MILES
CATEGORY,
Business,11487.0
Personal,717.7


In [94]:
DF['percent']=(DF['MILES'] / DF['MILES'].sum())*100
DF

,MILES,percent
CATEGORY,,
Business,11487.0,94.119479
Personal,717.7,5.880521


**Business miles percentage is 94 and Personal miles percentage are 6**

## Profile Report - 
Through the exercise, we discussed the various aspects the driver profile for the uberdriver data given. The insights received were-

**Name and number of all the unique start and stop points**<br>
We found the unique start and stop points for the driver. We figured out the localities the driver is active in.

**Popular start and stop point**<br>
Cary has been the most popular start and stop point

**Rides with same start and stop points**<br>
288 such rides were found

**Starting point from which most miles have been driven**<br>
Unknown location followed by Cary

**Start- stop pairs that are most travelled in terms of distance**<br>
Morissville - Cary

**busiest month in terms of number of drives and miles driven**<br>
In terms of no of drives - December
In terms of miles driven - October

**busiest day of the week**<br>
Friday

**peak hours**<br>
1PM - 6PM

**most frequent trip category**<br>
Business

**most frequent trip purpose**<br>
Meeting

**miles driven per category and purpose**<br>
We figured these numbers out. Category wise Business and purpose wise meetings were leading in terms of miles driven

**percent composition of business miles vs personal miles**<br>
Business - 94%
Personal - 6%


## Summary  -
Through this exercise, we tried to check out the data analysis toolkit offered by pandas. We went to explore variables at hand, use groupby, implement datatime manipulation, explored possibility to create new features and various other operations on pandas dataframe.
We also had a sneak peek into the upcoming week's topic of visualization.

Learners are recommended to explore further on this building on the points discussed in the notebook.
Happy Learning!